### Buoy Data
* Connecting to buoy data via csv url
* Used pandas dataframe to stage data

In [110]:
# Connection
import pandas as pd
import numpy as np
import re #importing regex for string parsing
pd.set_option('display.max_columns', None)

#reading website data as csv
buoy_data = pd.read_csv(
    'https://www.ndbc.noaa.gov/data/realtime2/45026.txt',
        
    sep = '\s+',
    header = 0,
    index_col = None
)

#dataframe to drop first record since first record represents units of measurements for fields within dataset
df = buoy_data.iloc[1:]
df1 = pd.DataFrame(df)

#### Manipulation

In [111]:
# renaming fields
df1.rename(columns = {
    '#YY':'Year','MM':'Month','DD':'Day','hh':'Hour','mm':'Minute','WDIR':'Wind_Dir','WSPD':'Wind_Speed','GST':'Wind_Gust','WVHT':'Wave_Height',
    'DPD':'Dominant_Wave_Prd','APD':'Average_Wave_Prd','MWD':'Dominant_Wave_Dir','PRES':'Sea_Level_Pres','ATMP':'Air_Temperature',
    'WTMP':'Wave_Temperature','DEWP':'Dew_Point_Temp','VIS':'Station_Visibility','PTDY':'Pressure_Tendency'
},inplace=True)

# Find and Replace all 'MM' values as null as this is likely a nullable value from source data - from observation
df1.replace(to_replace='MM',value = 0,inplace=True)

# appending datetime stamp field to existing df
df1.insert(19,'Timestamp',(df1['Year'] + '-' + df1['Month'] + '-' + df1['Day']+ ' ' + df1['Hour'] + ':' + df1['Minute']).astype('str'),True)
df1.insert(20,'yyyy-mm-dd',(df1['Year'] + '-' + df1['Month'] + '-' + df1['Day']).astype('str'))

#grouping and sorting dataframe by timestamp field| ensure most current record at top of file
df1.sort_values(by = 'Timestamp',ascending = False, inplace = True)

#############################################
#conversions
df1.insert(14,'Air_Temp_Deg_F',((df1['Air_Temperature'].astype('float')*(9/5))+32)) #converting air temp to fahrenheit
df1.insert(16,'Wave_Temp_Deg_F',((df1['Wave_Temperature'].astype('float')*(9/5))+32)) # converting wave temp to fahrenheit
df1.insert(18,'DewPoint_Temp_Deg_F',((df1['Dew_Point_Temp'].astype('float')*(9/5))+32)) # converting dewpoint temp to fahrenheit
df1.insert(7,'Wind_Speed_MPH',(df1['Wind_Speed'].astype('float')*2.237)) # converting windspeed to MPH
df1.insert(9,'Wind_Gust_MPH',(df1['Wind_Gust'].astype('float')*2.237)) # converting wind gust to MPH
df1.insert(11,'Wave_Height_FT',(df1['Wave_Height'].astype('float')*3.281)) # converting wave height to feet


###########################################
# appending condition for Wind and Wave Direction values
# compass direction dictionary
a = {
    'min_val':[0,11.25,33.75,56.25,78.75,101.25,123.75,146.25,168.75,191.25,213.75,236.25,258.75,281.25,303.75,326.25,348.75],
    'max_val':[11.24,33.74,56.24,78.24,101.24,123.74,146.24,168.74,191.24,213.74,236.24,258.74,281.24,303.74,326.24,348.74,360.00],
    'direction':['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW','N']
}
# changing to df
df_a = pd.DataFrame(a)

# function to apply compass direction to wave_direction data
def get_corresponding_value(i):
    for _, row in df_a.iterrows():
        if row['min_val'] <=i<=row['max_val']:
            return row['direction']
    return None

#appended values to df1 for wave direction
df1.insert(15,'Wave_Direction', df1['Dominant_Wave_Dir'].astype('float').apply(get_corresponding_value))
#appending values to df1 for wind direction
df1.insert(6,'Wind_Direction', df1['Wind_Dir'].astype('float').apply(get_corresponding_value))

#### Output Testing

In [112]:
df1.head(10)

,Year,Month,Day,Hour,Minute,Wind_Dir,Wind_Direction,Wind_Speed,Wind_Speed_MPH,Wind_Gust,Wind_Gust_MPH,Wave_Height,Wave_Height_FT,Dominant_Wave_Prd,Average_Wave_Prd,Dominant_Wave_Dir,Wave_Direction,Sea_Level_Pres,Air_Temperature,Air_Temp_Deg_F,Wave_Temperature,Wave_Temp_Deg_F,Dew_Point_Temp,DewPoint_Temp_Deg_F,Station_Visibility,Pressure_Tendency,TIDE,Timestamp,yyyy-mm-dd
1,2024,10,27,19,50,280,W,2.0,4.474,4.0,8.948,0.2,0.6562,0,0,316,NW,1026.2,12.4,54.32,16.6,61.88,2.8,37.04,0,0,0,2024-10-27 19:50,2024-10-27
2,2024,10,27,19,40,280,W,2.0,4.474,3.0,6.711,0.2,0.6562,0,0,327,NNW,1026.3,12.3,54.14,16.6,61.88,2.8,37.04,0,0,0,2024-10-27 19:40,2024-10-27
3,2024,10,27,19,30,270,W,2.0,4.474,4.0,8.948,0.2,0.6562,0,0,323,NW,1026.3,12.3,54.14,16.6,61.88,2.5,36.50,0,0,0,2024-10-27 19:30,2024-10-27
4,2024,10,27,19,20,260,W,2.0,4.474,4.0,8.948,0.2,0.6562,0,0,307,NW,1026.5,12.3,54.14,16.6,61.88,2.6,36.68,0,0,0,2024-10-27 19:20,2024-10-27
5,2024,10,27,19,10,250,WSW,3.0,6.711,4.0,8.948,0.2,0.6562,0,0,313,NW,1026.7,12.2,53.96,16.6,61.88,2.5,36.50,0,0,0,2024-10-27 19:10,2024-10-27
6,2024,10,27,19,00,250,WSW,3.0,6.711,4.0,8.948,0.2,0.6562,0,0,313,NW,1026.9,12.1,53.78,16.6,61.88,2.5,36.50,0,-3.0,0,2024-10-27 19:00,2024-10-27
7,2024,10,27,18,50,240,WSW,2.0,4.474,4.0,8.948,0.2,0.6562,0,0,327,NNW,1027.0,12.0,53.60,16.6,61.88,2.6,36.68,0,0,0,2024-10-27 18:50,2024-10-27
8,2024,10,27,18,40,250,WSW,3.0,6.711,4.0,8.948,0.2,0.6562,0,0,323,NW,1027.2,11.8,53.24,16.6,61.88,2.3,36.14,0,0,0,2024-10-27 18:40,2024-10-27
9,2024,10,27,18,30,250,WSW,2.0,4.474,3.0,6.711,0.2,0.6562,0,0,311,NW,1027.4,11.9,53.42,16.6,61.88,2.0,35.60,0,0,0,2024-10-27 18:30,2024-10-27
10,2024,10,27,18,20,260,W,2.0,4.474,3.0,6.711,0.2,0.6562,0,0,333,NNW,1027.8,11.8,53.24,16.6,61.88,2.0,35.60,0,0,0,2024-10-27 18:20,2024-10-27


#### CSV Output

In [113]:
# writing output to csv file 
df1.to_csv(r'C:/Users/errol/Documents/test_output.csv')